In [ ]:
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install PyPDF2
!pip install ipywidgets


In [ ]:
import os
import PyPDF2
from transformers import pipeline
from google.colab import files
from IPython.display import display
import ipywidgets as widgets


In [ ]:
print("Upload your text or PDF files containing notes:")
uploaded_files = files.upload()

notes_text = ""

def extract_text_from_pdf(path):
    text = ""
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text

for filename in uploaded_files.keys():
    if filename.endswith(".txt"):
        with open(filename, "r", encoding="utf-8") as f:
            notes_text += f.read() + "\n\n"
    elif filename.endswith(".pdf"):
        notes_text += extract_text_from_pdf(filename) + "\n\n"
    else:
        print(f"Skipped unsupported file: {filename}")

print("Notes uploaded successfully! Total characters:", len(notes_text))


In [ ]:
model_dropdown = widgets.Dropdown(
    options=['facebook/bart-large-cnn', 't5-small', 'google/pegasus-xsum'],
    value='facebook/bart-large-cnn',
    description='Model:',
)
display(model_dropdown)

summary_length_slider = widgets.IntRangeSlider(
    value=[40, 150],
    min=20,
    max=500,
    step=10,
    description='Summary Length:',
    continuous_update=False
)
display(summary_length_slider)


In [ ]:
def summarize_notes(model_name, min_len, max_len):
    summarizer = pipeline("summarization", model=model_name)

    # Split into chunks if text is long
    max_chunk_size = 1000
    chunks = [notes_text[i:i+max_chunk_size] for i in range(0, len(notes_text), max_chunk_size)]

    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    final_summary = " ".join(summaries)
    return final_summary


In [ ]:
start_button = widgets.Button(description="Generate Summary")
output_area = widgets.Output()
display(start_button, output_area)

def on_button_click(b):
    with output_area:
        output_area.clear_output()
        print("Generating summary...")
        model_name = model_dropdown.value
        min_len, max_len = summary_length_slider.value
        final_summary = summarize_notes(model_name, min_len, max_len)

        # Show preview of original notes
        print("\n--- Original Notes Preview ---\n")
        print(notes_text[:1000], "...\n")

        # Show summarized notes
        print("\n--- Summarized Notes ---\n")
        print(final_summary)

        # Save summary to file
        with open("summarized_notes.txt", "w", encoding="utf-8") as f:
            f.write(final_summary)
        files.download("summarized_notes.txt")
        print("\n✅ Summary saved and downloaded as 'summarized_notes.txt'!")

start_button.on_click(on_button_click)
